In [2]:
## Initial setup
import qiskit
from qiskit_ibm_runtime import QiskitRuntimeService
import os
from dotenv import load_dotenv

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")
service = QiskitRuntimeService(channel="ibm_quantum", token=API_TOKEN)
backend = service.backend(name="ibm_rensselaer")

In [3]:
# Initialize 2 qubits to 0
qc = qiskit.QuantumCircuit(2)
qc.initialize('00')
qc.measure_all()

In [5]:
# Execute the circuit
qc_transpiled = qiskit.transpile(qc, backend)
job = backend.run(qc_transpiled, shots=10**4)
result = job.result()

/tmp/ipykernel_124609/864542522.py:3: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job = backend.run(qc_transpiled, shots=10**4)


In [6]:
counts = result.get_counts()
print(counts)

{'10': 69, '01': 4, '00': 9927}


In [ ]:
# Now, try with 20 qubits
qc = qiskit.QuantumCircuit(20)
qc.initialize('0'*20)
qc.measure_all()

# Execute the circuit
qc_transpiled = qiskit.transpile(qc, backend)
job = backend.run(qc_transpiled, shots=10**4)
result = job.result()



/tmp/ipykernel_124609/1135121357.py:8: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job = backend.run(qc_transpiled, shots=10**4)


{'00001000000000000001': 1, '10000000100000000000': 1, '00000000110000000000': 24, '00000101001000000000': 1, '00000000001000000000': 256, '00000000001000000100': 1, '00000100000101000000': 1, '00000000000000010000': 30, '00001000000000001000': 1, '00010001100000000000': 2, '00010000000000100000': 1, '00110001000000000000': 2, '00000000000100100000': 2, '01000010000000000000': 1, '00000000000010000000': 18, '01000000000000000010': 1, '00100000010000010000': 1, '00000000000000000001': 6, '00110000000000010000': 1, '01000000000000000000': 96, '00000101000000000000': 2, '00011000000000000000': 1, '00000000000000000010': 72, '00000000100000100000': 1, '01000000010000000000': 2, '00001000000001000000': 2, '00000000000010000001': 1, '00000000000101000000': 2, '00100000000000100000': 1, '00000000100010000000': 1, '00010000100000000100': 1, '00101001000000000000': 1, '00000100001000000000': 1, '00101000000000000000': 7, '00000001000000000000': 314, '00100000000000010000': 3, '01000001000000000

In [8]:
counts = result.get_counts()
print(counts)
print(counts['0'*20])

{'00001000000000000001': 1, '10000000100000000000': 1, '00000000110000000000': 24, '00000101001000000000': 1, '00000000001000000000': 256, '00000000001000000100': 1, '00000100000101000000': 1, '00000000000000010000': 30, '00001000000000001000': 1, '00010001100000000000': 2, '00010000000000100000': 1, '00110001000000000000': 2, '00000000000100100000': 2, '01000010000000000000': 1, '00000000000010000000': 18, '01000000000000000010': 1, '00100000010000010000': 1, '00000000000000000001': 6, '00110000000000010000': 1, '01000000000000000000': 96, '00000101000000000000': 2, '00011000000000000000': 1, '00000000000000000010': 72, '00000000100000100000': 1, '01000000010000000000': 2, '00001000000001000000': 2, '00000000000010000001': 1, '00000000000101000000': 2, '00100000000000100000': 1, '00000000100010000000': 1, '00010000100000000100': 1, '00101001000000000000': 1, '00000100001000000000': 1, '00101000000000000000': 7, '00000001000000000000': 314, '00100000000000010000': 3, '01000001000000000